In [1]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet34
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset, read_pred_csv
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

from torch import Tensor
from collections import OrderedDict, defaultdict
import copy

import os

## Prepare Data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [2]:
DEBUG = False

# training cfg
validation_cfg = {
    
    'format_version': 4,
    
     ## Model options
    'model_params': {
        'model_architecture': 'resnet34',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
    },

    ## Input raster parameters
    'raster_params': {
        
        'raster_size': [224, 224], # raster's spatial resolution [meters per pixel]: the size in the real world one pixel corresponds to.
        'pixel_size': [0.5, 0.5], # From 0 to 1 per axis, [0.5,0.5] would show the ego centered in the image.
        'ego_center': [0.25, 0.5],
        'map_type': "py_semantic",
        
        # the keys are relative to the dataset environment variable
        'satellite_map_key': "aerial_map/aerial_map.png",
        'semantic_map_key': "semantic_map/semantic_map.pb",
        'dataset_meta_key': "meta.json",

        # e.g. 0.0 include every obstacle, 0.5 show those obstacles with >0.5 probability of being
        # one of the classes we care about (cars, bikes, peds, etc.), >=1.0 filter all other agents.
        'filter_agents_threshold': 0.5
    },

    ## Data loader options
    'valid_data_loader': {
        'key': "scenes/validate_chopped_100/validate.zarr",
        'batch_size': 1,
        'shuffle': False,
        'num_workers': 0
    },

    ## Valid params
    'valid_params': {
        'checkpoint_every_n_steps': 5000,
        'max_num_steps': 10 if DEBUG else 1000
    }
}

common_cfg = {
    'seed': 500,
    'output_dir': './outputs/1011_2/',
    'epoch': 2,
    'train_step': 5 if DEBUG else 500,
    'valid_step': 5 if DEBUG else 50,
    'train_max': 12,
    'learning_rate': 1e-3
}


In [3]:
OUTPUT_DIR = common_cfg['output_dir']
INPUT_ROOT = Path('/home/knikaido/work/Lyft/data/')
DATA_DIR = INPUT_ROOT / 'lyft-motion-prediction-autonomous-vehicles/'

In [4]:
pred_path1 = f"{OUTPUT_DIR}/pred_1010_custom.csv"
pred_path2 = f"{OUTPUT_DIR}/pred_1114.csv"

In [5]:
eval_gt_path = str(f"{str(DATA_DIR)}/scenes/validate_chopped_100/gt.csv")

In [6]:
pred_pathes = [pred_path1, pred_path2]
outputs = []
for path in pred_pathes:
    output = pd.read_csv(path)
    output.sort_values(['timestamp', 'track_id'], inplace=True)
    output.reset_index(inplace=True, drop=True)
    outputs.append(output)


In [7]:
t_predicts = []
t_confs = []

In [8]:
for i in tqdm(range(94694)):
    predicts = []
    confs = []
    confs.append(outputs[0].loc[i, 'conf_0'])
    confs.append(outputs[0].loc[i, 'conf_1'])
    confs.append(outputs[0].loc[i, 'conf_2'])
    predicts.append(outputs[0].loc[i, 'coord_x00':'coord_y049'].values)
    predicts.append(outputs[0].loc[i, 'coord_x10':'coord_y149'].values)
    predicts.append(outputs[0].loc[i, 'coord_x20':'coord_y249'].values)
    confs.append(outputs[1].loc[i, 'conf_0'])
    confs.append(outputs[1].loc[i, 'conf_1'])
    confs.append(outputs[1].loc[i, 'conf_2'])
    confs.append(outputs[1].loc[i, 'conf_3'])
    confs.append(outputs[1].loc[i, 'conf_4'])
    confs.append(outputs[1].loc[i, 'conf_5'])
    confs.append(outputs[1].loc[i, 'conf_6'])
    predicts.append(outputs[1].loc[i, 'coord_x00':'coord_y049'].values)
    predicts.append(outputs[1].loc[i, 'coord_x10':'coord_y149'].values)
    predicts.append(outputs[1].loc[i, 'coord_x20':'coord_y249'].values)
    predicts.append(outputs[1].loc[i, 'coord_x30':'coord_y349'].values)
    predicts.append(outputs[1].loc[i, 'coord_x40':'coord_y449'].values)
    predicts.append(outputs[1].loc[i, 'coord_x50':'coord_y549'].values)
    predicts.append(outputs[1].loc[i, 'coord_x60':'coord_y649'].values)
    t_predicts.append(predicts)
    t_confs.append(confs)

100%|██████████| 94694/94694 [08:47<00:00, 179.58it/s]


In [9]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


In [10]:
def calc_distance(coord1, coord2):
    coord1 = coord1.reshape(-1, 2)
    coord2= coord2.reshape(-1, 2)
    distance = 0
    cos = 0
    for i in range(len(coord1)):
        cos += cos_sim(coord2[i], coord1[i])
        distance += pow(coord2[i, 0] - coord1[i, 0], 2) + pow(coord2[i, 1] - coord1[i, 1], 2)
        
    return cos / len(coord1), np.sqrt(distance)
    

In [11]:
def decide_preds(tmp_predicts, tmp_conf, thre=30):
    
    
    predicts = copy.deepcopy(tmp_predicts)
    confs = copy.deepcopy(tmp_conf)
    
    ans_preds = []
    ans_confs = []
    
    for i in range(3):
        ind = np.argmax(confs)
        pre1 = predicts[ind]
        ans_preds.append(pre1)
        ans_confs.append(confs[ind])
        predicts.pop(ind)
        confs.pop(ind)

        predicts_ = []
        confs_ = []
        for i in range(len(predicts)):
            angle, dist = calc_distance(pre1, np.array(predicts[i]))
            if(dist > thre):
                predicts_.append(predicts[i])
                confs_.append(confs[i])
                
        if(len(confs_)<3):
            return tmp_predicts[0:3], tmp_conf[0:3] / np.sum(tmp_conf[0:3])
            
        predicts = predicts_
        confs = confs_
        
    ans_confs = ans_confs / np.sum(ans_confs)
    return ans_preds, ans_confs
        

In [12]:
for i in range(len(t_confs)):
    for j in range(3):
#         t_confs[i][j] = t_confs[i][j] * 2  
        t_confs[i][j] = t_confs[i][j] * (21.3 / 15.3)  

In [13]:
ans_output = pd.read_csv(str(OUTPUT_DIR)+'pred_1010.csv')
ans_output.sort_values(['timestamp', 'track_id'], inplace=True)
ans_output.reset_index(inplace=True, drop=True)
ans_conf = ans_output.loc[:, 'conf_0':'conf_2']
ans_coord= ans_output.loc[:, 'coord_x00':]

In [14]:
for j in tqdm(range(len(ans_output))):
    prepre, concon = decide_preds(t_predicts[j], t_confs[j])

    ans_conf.iloc[j] = concon
    ans_coord.iloc[j] = np.array(prepre).reshape(-1)


100%|██████████| 94694/94694 [17:31<00:00, 90.09it/s]


In [15]:
ans_output.loc[:, 'conf_0':'conf_2'] = ans_conf
ans_output.loc[:, 'coord_x00':] = ans_coord

In [16]:
ans_output.to_csv(str(OUTPUT_DIR)+'pred_ansamble.csv', index=False)
pred_path_ = str(OUTPUT_DIR)+'pred_ansamble.csv'
ans_output

,timestamp,track_id,conf_0,conf_1,conf_2,coord_x00,coord_y00,coord_x01,coord_y01,coord_x02,...,coord_x245,coord_y245,coord_x246,coord_y246,coord_x247,coord_y247,coord_x248,coord_y248,coord_x249,coord_y249
0,1574364399902601396,1,0.452226,0.174998,0.372776,0.352950,0.768407,0.703219,1.500859,1.054638,...,14.039252,25.233285,14.249384,25.522800,14.455089,25.805094,14.659608,26.085474,14.866672,26.369638
1,1574364399902601396,3,0.423236,0.182002,0.394762,0.432038,0.845263,0.846765,1.688959,1.261487,...,14.602966,29.482602,14.833410,29.927132,15.058813,30.361709,15.282684,30.792851,15.508991,31.228273
2,1574364424902827486,2,0.419648,0.186097,0.394255,0.681790,1.085086,1.404457,2.185226,2.126384,...,31.973179,48.609641,32.627200,49.608317,33.273555,50.597053,33.919056,51.585424,34.571631,52.584600
3,1574364449901934626,836,0.506082,0.322601,0.171317,-0.335677,-0.449166,-0.687134,-0.913975,-1.038315,...,-13.896561,-15.387691,-14.151020,-15.598765,-14.398628,-15.808804,-14.643038,-16.020653,-14.888909,-16.236923
4,1574364474902117814,1,0.514791,0.206952,0.278257,0.556707,0.942090,1.145457,1.933617,1.730301,...,14.984266,45.990126,14.995755,47.001849,14.997587,47.999403,14.994586,48.993612,14.991583,49.997492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94689,1584140351702477836,8,0.313687,0.095161,0.591152,-0.571036,-0.948232,-1.178492,-1.936991,-1.787196,...,-31.030810,-48.008019,-31.725901,-49.048264,-32.413619,-50.078583,-33.100743,-51.108820,-33.795191,-52.150332
94690,1584140451301713374,364,0.433969,0.165961,0.400070,0.890398,1.391600,1.822207,2.812443,2.753794,...,40.081914,60.863457,40.876519,62.075109,41.658724,63.268873,42.437688,64.458029,43.224263,65.658755
94691,1584140451301713374,494,0.617809,0.234368,0.147823,-0.352561,-0.018846,-0.516613,0.160633,-0.680741,...,-33.109600,-7.008330,-33.748150,-7.868110,-34.363570,-8.413680,-35.035580,-9.470680,-35.616540,-10.119570
94692,1584140476302704606,1,0.628066,0.275751,0.096183,-0.377475,-0.574794,-0.741325,-1.154883,-1.106062,...,-22.529840,-13.419110,-23.074910,-13.442330,-23.607780,-13.377140,-24.136200,-13.425480,-24.685550,-13.331640


In [17]:
metrics = compute_metrics_csv(eval_gt_path, pred_path_, [neg_multi_log_likelihood, time_displace])
for metric_name, metric_mean in metrics.items():
    print(metric_name, metric_mean)

neg_multi_log_likelihood 25.66120067017779
time_displace [0.05442364 0.07579905 0.09613111 0.11560009 0.13850711 0.15862505
 0.18074003 0.19815278 0.21751999 0.23756573 0.25637135 0.27776774
 0.29794042 0.31682845 0.33442899 0.35163166 0.36747446 0.38359619
 0.39948102 0.41210166 0.42525736 0.43870085 0.45107895 0.46337037
 0.47534243 0.4871324  0.49869956 0.51045509 0.52109936 0.53023118
 0.54081849 0.55030255 0.5610502  0.570272   0.58113853 0.59325087
 0.60446603 0.61737562 0.62916133 0.64250092 0.6570162  0.6711426
 0.68577761 0.70147195 0.71436816 0.73171403 0.74988826 0.76932899
 0.79029229 0.81173815]


In [47]:
neg_multi_log_likelihood 21.234776685153225

SyntaxError: invalid syntax (<ipython-input-47-24cc5f676833>, line 1)

In [ ]:
26.188473004283985